In [38]:
import os

import numpy as np
import operator
from termcolor import colored
import pandas as pd
import random
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

import nltk
#from nltk.stem.wordnet import WordNetLemmatizer 
from nltk.stem import WordNetLemmatizer 

from nltk import word_tokenize, pos_tag

from nltk.corpus import stopwords

import collections
import multiprocessing as mp
import re

import sklearn.datasets
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

from spacy.lang.en import English


In [2]:
data_directory = 'reviews_dataset'
movie_sentiment_data = sklearn.datasets.load_files(data_directory, shuffle=True)

In [45]:
#word tokenizer
nlp = English()

In [4]:
print("files loaded {}".format(len(movie_sentiment_data.data)))
print("they contain the following classes {}".format(movie_sentiment_data.target_names))

files loaded 2000
they contain the following classes ['neg', 'pos']


In [46]:
# preprocessing
def extract_features(corpus):
    count_vectorizer = sklearn.feature_extraction.text.CountVectorizer(
        lowercase = True,
        tokenizer = nlp, #nltk.word_tokenize,# nlp
        min_df = 1,
        ngram_range = (1,3),
        stop_words = 'english'
    )
    
    processed_corpus = count_vectorizer.fit_transform(corpus)
    obj = sklearn.feature_extraction.text.TfidfTransformer()
    processed_corpus = obj.fit_transform(processed_corpus)
    
    return processed_corpus, obj, count_vectorizer

In [6]:
print(nltk.word_tokenize("hoiii hello"))

['hoiii', 'hello']


In [47]:
movie_tfdif, obj, cv = extract_features(movie_sentiment_data.data)
#n_gram, obj, cv = extract_features(movie_sentiment_data.data) 

/home/abhiram/anaconda3/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens [put, yet, although, me, further, otherwise, throughout, be, etc, while, again, detail, too, one, though, besides, mostly, by, thence, nevertheless, hers, somehow, this, made, perhaps, meanwhile, go, then, ca, still, we, the, bill, next, system, eight, thereby, before, why, so, amount, whom, see, somewhere, therein, afterwards, nothing, thereafter, everyone, several, below, serious, indeed, co, beyond, six, whenever, last, seems, after, always, without, name, elsewhere, cry, neither, same, latter, call, five, some, he, here, down, else, a, if, anyone, due, beside, of, however, to, were, often, most, anyhow, front, fire, wherever, whether, anywhere, up, out, would, into, must, once, moreover, least, please, such, whereby, other, both, what, sometimes, become, well, from, becomes, t

TypeError: sequence item 0: expected str instance, spacy.tokens.token.Token found

In [70]:
obj.idf_

array([7.21510797, 7.50279005, 2.10312019, ..., 7.21510797, 7.50279005,
       7.50279005])

In [27]:
movie_tfdif[0].toarray()

array([[0., 0., 0., ..., 0., 0., 0.]])

In [22]:
movie_sentiment_data.target_names

['neg', 'pos']

In [7]:
#pd.DataFrame(obj.idf_, index=cv.get_feature_names(),columns=["idf_weights"]).to_csv('ifdf_updated_values.csv')
pd.DataFrame(obj.idf_, index=cv.get_feature_names(),columns=["idf_weights"])

,idf_weights
,7.908255
 humour,7.908255
 humour !,7.908255
earth,7.908255
earth man,7.908255
"earth man ,",7.908255
goodies,7.908255
goodies versus,7.908255
goodies versus baddies,7.908255
if,7.908255


In [42]:
spacy_token = pd.DataFrame(obj.idf_, index=cv.get_feature_names(),columns=["idf_weights"])

In [45]:
nltk_token = pd.DataFrame(obj.idf_, index=cv.get_feature_names(),columns=["idf_weights"])

In [46]:
spacy_token.shape, nltk_token.shape

((1619080, 1), (46169, 1))

In [52]:
[print(data) for data in movie_sentiment_data.data]

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)




b'synopsis : original " jurassic park " survivor alan grant ( neill ) is tricked by paul and amanda kirby ( macy and leoni ) into leading an expedition to an island overrun by dinosaur , to rescue their stranded son . \ngrant\'s team is picked off one by one as they encounter dinosaurs both new and familiar , and grant grows to realise that some breeds are more intelligent than he previously thought . \nreview : there is one reason why " jurassic park iii " is better than its immediate predecessor , " the lost world : jurassic park " . \nthe latter wasted 129 minutes of my life ; the former , only 94 . \nthe first " park " was a marvellous film , full of awe-inspiring sights , interesting characters , and genuine thrills . \nits two successors have been pale imitations , amounting to hundred-million-dollar b-grade horror movies . \nat least this third entry realises it ; whereas " the lost world " tried to justify its existence with a ludicrous and overextended plot , " jurassic park 

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [15]:
x_train, x_test, y_train, y_test = train_test_split(movie_tfdif, movie_sentiment_data.target, 
                                                    test_size= 0.3, random_state = 42)

In [16]:
model = LogisticRegression()
model.fit(x_train, y_train)

print("model performance {}".format(model.score(x_test, y_test)))

model performance 0.7433333333333333


In [17]:
x_train.todense()[2].shape

(1, 1245724)

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [18]:
y_pred = model.predict(x_test)

for i in range(5):
    print('Review: \n{review}\n-\n Correct label: {correct}; Predicted: {predicted} '.format(
                    review = x_test[i], correct = y_test[i], predicted = y_pred[i]))

Review: 
  (0, 1776)	0.016591961425870272
  (0, 1827)	0.038307675289264126
  (0, 1828)	0.038307675289264126
  (0, 2089)	0.016873669345270424
  (0, 2123)	0.038307675289264126
  (0, 2124)	0.038307675289264126
  (0, 2412)	0.009489008473300302
  (0, 4738)	0.038307675289264126
  (0, 4739)	0.038307675289264126
  (0, 32089)	0.06620053593177647
  (0, 32093)	0.030049856706585446
  (0, 32096)	0.038307675289264126
  (0, 33090)	0.036343597828431204
  (0, 33092)	0.038307675289264126
  (0, 33131)	0.0322392776368291
  (0, 33133)	0.038307675289264126
  (0, 33374)	0.03298598523204842
  (0, 33376)	0.036343597828431204
  (0, 33550)	0.036343597828431204
  (0, 33552)	0.038307675289264126
  (0, 33784)	0.03386915286635803
  (0, 33785)	0.03495006269288135
  (0, 36419)	0.038307675289264126
  (0, 36420)	0.038307675289264126
  (0, 36440)	0.03495006269288135
  :	:
  (0, 1160990)	0.03495006269288135
  (0, 1160992)	0.038307675289264126
  (0, 1164473)	0.013924374160322755
  (0, 1164781)	0.030511540269975247
  (0, 11

import spacy
from dframcy import DframCy

nlp = spacy.load('en_core_web_sm')
dframcy = DframCy(nlp)
doc = dframcy.nlp(u'Apple is looking at buying U.K. startup for $1 billion')
annotation_dataframe = dframcy.to_dataframe(doc)

In [2]:
#!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.0.0/en_core_web_sm-2.0.0.tar.gz